In [1]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pyspark
from pyspark.sql import SparkSession
import os
from IPython.core.display import HTML

In [3]:
## DEFINE SENSITIVE VARIABLES
NESSIE_URI = os.environ.get("NESSIE_URI") ## Nessie Server URI
WAREHOUSE = os.environ.get("WAREHOUSE") ## BUCKET TO WRITE DATA TOO
MINIO_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY") ## AWS CREDENTIALS
MINIO_SECRET_ACCESS_KEY = os.environ.get("MINIO_SECRET_ACCESS_KEY") ## AWS CREDENTIALS
MINIO_S3_ENDPOINT= os.environ.get("MINIO_S3_ENDPOINT") ## MINIO ENDPOINT

os.environ["AWS_ACCESS_KEY_ID"] = MINIO_ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = MINIO_SECRET_ACCESS_KEY
os.environ["AWS_S3_ENDPOINT"] = MINIO_S3_ENDPOINT

print(MINIO_S3_ENDPOINT)
print(NESSIE_URI)
print(WAREHOUSE)

http://minioserver:9000
http://nessie:19120/api/v1
s3a://warehouse/


In [4]:
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.3,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.76.3,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.s3.endpoint', MINIO_S3_ENDPOINT)
        .set('spark.sql.catalog.nessie.warehouse', WAREHOUSE)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_ACCESS_KEY)
)


## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/home/docker/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/docker/.ivy2/cache
The jars for the packages stored in: /home/docker/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6f60c840-b9d1-45da-ba30-c6098d1410c1;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.3 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.76.3 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central

Spark Running


In [5]:
df = spark.read.parquet(f"./data/flights_data5_develop_sample_2019-09-01.snappy.parquet")

In [8]:
display(df.toPandas())

,icao24,firstseen,estdepartureairport,lastseen,estarrivalairport,callsign,estdepartureairporthorizdistance,estdepartureairportvertdistance,estarrivalairporthorizdistance,estarrivalairportvertdistance,departureairportcandidatescount,arrivalairportcandidatescount,day,airportofdeparture,airportofdestination,takeofftime
0,4247ad,1.567351e+09,LPCS,1567358944,EBMB,VPCBD,1208.0,243.0,1557.0,23.0,1,3,1567296000,LPCS,LFPB,NaN
1,44d2b8,1.567363e+09,LGRP,1567374878,EBMB,JAF5EJ,872.0,40.0,1062.0,7.0,1,3,1567296000,LGRP,EBBR,NaN
2,44d2b8,1.567346e+09,EBBR,1567357917,LGRP,JAF77Y,1377.0,10.0,528.0,48.0,2,1,1567296000,EBBR,LGRP,1.567346e+09
3,44d2b8,1.567330e+09,None,1567341193,EBMB,JAF78M,NaN,NaN,3386.0,38.0,0,3,1567296000,LGSA,EBBR,NaN
4,44d2b8,1.567311e+09,EBBR,1567321429,LGSA,JAF2PQ,856.0,17.0,25919.0,1603.0,2,0,1567296000,EBBR,LGSA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,3461c1,1.567360e+09,EBLG,1567368311,LEMD,ECNCJ,1164.0,279.0,1113.0,37.0,0,3,1567296000,LECU,LEMD,NaN
948,4848df,1.567354e+09,EGPF,1567358902,EHRD,PHTXA,776.0,213.0,684.0,41.0,0,1,1567296000,EBAW,EHRD,1.567354e+09
949,4a5321,1.567350e+09,EGKB,1567352757,EBLG,TOY103,521.0,46.0,10201.0,408.0,0,4,1567296000,EGKB,EBLG,1.567350e+09
950,4ca815,1.567360e+09,EBCI,1567364698,LIPM,RYR5MU,4876.0,513.0,19214.0,1018.0,2,2,1567296000,EBCI,LIPE,NaN
